In [1]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
trainDir = "/Users/ppr/Desktop/Project/Intern-Year2/Datasets/GrayDatasetsResizeAugmented/train"
valDir = "/Users/ppr/Desktop/Project/Intern-Year2/Datasets/GrayDatasetsResizeAugmented/val"
testDir = "/Users/ppr/Desktop/Project/Intern-Year2/Datasets/GrayDatasetsResizeAugmented/test"

In [3]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(640, 640, 3))

In [4]:
for layer in base_model.layers:
    layer.trainable = False

In [5]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(11, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    trainDir,
    target_size=(640, 640),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    valDir,
    target_size=(640, 640),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical'
)

Found 781 images belonging to 11 classes.
Found 147 images belonging to 11 classes.


In [12]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 640, 640, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 640, 640, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 640, 640, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 320, 320, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 320, 320, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 320, 320, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 160, 160, 128)     0     

In [8]:
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=50,
    validation_data=val_generator,
    validation_steps=len(val_generator)
)

Epoch 1/10


2023-05-31 03:09:36.390219: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


25/25 [==============================] - ETA: 0s - loss: 2.2245 - accuracy: 0.2561 

2023-05-31 03:18:02.829712: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


25/25 [==============================] - 598s 24s/step - loss: 2.2245 - accuracy: 0.2561 - val_loss: 2.1353 - val_accuracy: 0.3469
Epoch 2/10
25/25 [==============================] - 587s 24s/step - loss: 1.8813 - accuracy: 0.4020 - val_loss: 1.8405 - val_accuracy: 0.3605
Epoch 3/10
25/25 [==============================] - 609s 24s/step - loss: 1.6440 - accuracy: 0.4712 - val_loss: 1.7340 - val_accuracy: 0.3333
Epoch 4/10
25/25 [==============================] - 618s 25s/step - loss: 1.4826 - accuracy: 0.5429 - val_loss: 1.6386 - val_accuracy: 0.4354
Epoch 5/10
25/25 [==============================] - 614s 25s/step - loss: 1.3696 - accuracy: 0.5480 - val_loss: 1.5501 - val_accuracy: 0.4966
Epoch 6/10
25/25 [==============================] - 603s 24s/step - loss: 1.2607 - accuracy: 0.6031 - val_loss: 1.4142 - val_accuracy: 0.5442
Epoch 7/10
25/25 [==============================] - 565s 23s/step - loss: 1.1659 - accuracy: 0.6351 - val_loss: 1.3675 - val_accuracy: 0.5850
Epoch 8/10
25/25 

In [9]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    testDir,
    target_size=(640, 640),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 133 images belonging to 11 classes.


In [10]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_generator))
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

2023-05-31 05:26:16.334310: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


5/5 [==============================] - 189s 35s/step - loss: 1.0494 - accuracy: 0.6316
Test Loss: 1.049368977546692
Test Accuracy: 0.6315789222717285


In [11]:
model.save('VGG16Model-1.h5')

/Users/ppr/miniforge3/envs/sophomore/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
model.save('VGG16Model-1.keras')